In [1]:
%pip install "scikit-learn>=1.7.2" \
             "numpy<=2.2"

%pip install "ipykernel>=6.30.1"

/home/nic/local-code/dsci-789/dsci-89/.venv/bin/python: No module named pip
Note: you may need to restart the kernel to use updated packages.
/home/nic/local-code/dsci-789/dsci-89/.venv/bin/python: No module named pip
Note: you may need to restart the kernel to use updated packages.


First we are training a complex modelon the California housing data and evaluating the model.

In [2]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error, r2_score
import numpy as np

# Load data
data = fetch_california_housing(as_frame=True)

X = data.data
y = data.target

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=0
)

# Define and Train model
model = RandomForestRegressor()
model.fit(X_train, y_train)
    
# Evaluate Model
y_pred = model.predict(X_test)
rmse = root_mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"RMSE: {rmse:.3f}, R²: {r2:.3f}")

RMSE: 0.514, R²: 0.798


Next, we sample an instance from the test data with some constraints. The constraints are arbitrary; they are just so we can select a instance in a specific range that we are interested in. This is the instance we will explain.

In [3]:
def get_random_instance(y, X_sub, model, percentile=0.95, 
                        above=True, random_state=0):
    
    # Calculate value at given percentile
    p = y.quantile(percentile)
    # Select data from above or below the percentile threshold
    subset = y[y >= p] if above else y[y <= p]
    # Randomly sample from the subet
    random_instance = subset.sample(n=1, random_state=random_state)
    
    # Extract the index and features of the chosen sample
    idx = random_instance.index[0]
    x_instance = X_sub.loc[[idx]]
    
    # Extract true and predicted values of the sample
    true_val = random_instance.values[0]
    pred_val = model.predict(x_instance)[0]
    
    # Zip features with labels for easy identificaiton later
    features = zip(x_instance.columns, x_instance.values[0])

    return {
        "idx": idx,
        "true_val": true_val,
        "pred_val": pred_val,
        "percentile_value": p,
        "features": features,
        "x": x_instance
    }
    
def show_random_instance(instance_dict, percentile):
    print(f"\n{percentile}th percentile of y_test: {instance_dict['percentile_value']:.3f}")
    print(f"Random Instance:")
    print(f"  Index: {instance_dict['idx']}")
    print(f"  True value: {instance_dict['true_val']:.3f}")
    print(f"  Predicted value: {instance_dict['pred_val']:.3f}")
    print(f"  Features")
    for feature in instance_dict["features"]:
        print(f"   {feature[0]}:{feature[1]}")
   

local_instance = get_random_instance(y_test, X_test, model, 
                                     percentile=0.95, above=True)
show_random_instance(local_instance, percentile=95)


95th percentile of y_test: 4.764
Random Instance:
  Index: 5419
  True value: 5.000
  Predicted value: 3.894
  Features
   MedInc:3.9091
   HouseAge:38.0
   AveRooms:5.9021739130434785
   AveBedrms:1.1875
   Population:830.0
   AveOccup:2.255434782608696
   Latitude:34.02
   Longitude:-118.43


Now, we follow a manual process to approximate the LIME approach. First, we generate a neighborhood of synthetic points around the sample. 
I have chosen to use a KNN approach to this where we:

- Find the 50 nearest REAL data points from the training data using a KNN model
- Sample them with replacement 1000 times
- Take that set and perturb them slightly with noise so they are similar to the 50 real data points but not the same.

This will hopefully help keep the features more realistic than pure noise based approach, since we will not accidentally create a possible but implausible points like houses with 10 rooms but only 1 bedroom or houses in areas with high population but very low occupancy.

In [4]:
from sklearn.neighbors import NearestNeighbors

def sample_neighbors(x, X_train, num_samples=1000, n_neighbors=50,
                         jitter=True, jitter_scale=0.01):


    # Fit a nearest neighbor model to the training data
    nn = NearestNeighbors(n_neighbors=n_neighbors).fit(X_train.to_numpy())

    # Find nearest neighbors
    x = np.array(x).reshape(1, -1)
    _, indices = nn.kneighbors(x)
    candidate_neighbors = X_train.iloc[indices[0]]

    # Sample those neighbors with replacement 
    sampled = candidate_neighbors.sample(n=num_samples, replace=True,
                                         random_state=0).to_numpy()

    # The points will overlap so we can add some noise to them
    if jitter:
        std = X_train.std(axis=0)
        # The jitter_scale helps control how far we're moving the point 
        # its only 1% of the STD based on my defaults
        noise = np.random.normal(scale=std * jitter_scale, size=sampled.shape)
        sampled = sampled + noise

        # Clip so samples stay in valid range
        X_min, X_max = X_train.min(axis=0).to_numpy(), X_train.max(axis=0).to_numpy()
        sampled = np.clip(sampled, X_min, X_max)

    return sampled


perturbed_data = sample_neighbors(local_instance["x"], X_train)

print(perturbed_data.shape)

(1000, 8)


Next we calculate the distance of each of the perturbed samples from the sample we want to explain. I am using Euclidian distance.

We cannot use the distances directly because further data points have higher values which is the opposite of what we want. The LIME paper specified a gaussion kernel

$$K(x, z) = \exp(-\frac{||x-z||^2}{\sigma^2})$$

I am using the default for the kernel width :  $\sigma = 0.75 \times \sqrt{\text{}num features}$

In [5]:
def euclidian_dist(point1, point2):
    difference = point2 - point1
    return np.linalg.norm(difference)

distances = []
for x in perturbed_data:
    distances.append(euclidian_dist(local_instance["x"], x))


kernel_width = 0.75 * np.sqrt(perturbed_data.shape[1])
weights = np.exp(-(np.array(distances) ** 2) / (kernel_width ** 2))

Finally, we use the original model to predict values for the perturbed data. 

We then fit a new linear model using the perturbed data, the predictions from the original black-box model and the weights.

The coefficients of this model are our explanation weights. I have zipped them with their feature names, sorted them by abosolute value and printed them here.

In [6]:
import warnings
warnings.filterwarnings("ignore", message="X does not have valid feature names") # supressing this warning because it doesn't amtter for what we're doing

neighbor_preds = model.predict(perturbed_data)
    
local_model = LinearRegression()
local_model.fit(perturbed_data, neighbor_preds, sample_weight=weights)
explanation = dict(zip(X_train.columns, local_model.coef_))

explanation = dict(sorted(explanation.items(), key=lambda x: abs(x[1]), reverse=True))

for feat, weight in explanation.items():
    print(f"{feat}: \t{weight:.4f}")


AveBedrms: 	4.1582
AveOccup: 	-0.6961
MedInc: 	0.4718
Latitude: 	-0.1577
Longitude: 	0.1315
HouseAge: 	0.1152
AveRooms: 	-0.0357
Population: 	0.0056
